# Chapter 2: Reading and Writing Files

## Reading and writing CSV files

### Getting ready

### How to do it...

### There is more...

## Reading and writing parquet files

## Reading and writing CSV files

## Reading and writing CSV files

## Reading and writing CSV files

## Reading and writing CSV files

## Reading and writing CSV files